# Class 5 - Neural networks and Deep Learning

## Fashion-MNIST classification

### Data load

In [ ]:
#Download Fashion-MNIST repository/data
!rm -rf fashion-mnist/
!git clone https://github.com/zalandoresearch/fashion-mnist.git

In [ ]:
!gunzip fashion-mnist/data/fashion/*

In [ ]:
!mv fashion-mnist/data/fashion/* .

In [ ]:
#Function - convertion of binary input to .csv
def convertbinMNIST(img, label, output, n):
    fimg = open(img, "rb")
    fout = open(output, "w")
    flabel = open(label, "rb")
    
    #Get rid of headers
    fimg.read(16)
    flabel.read(8)
    mod = n/10
    print("Progress: ", end='')
    
    for i in range(n):
        image = [ord(flabel.read(1))]
        for j in range(28*28):
            image.append(ord(fimg.read(1)))
        fout.write(",".join(str(pix) for pix in image)+"\n")
        if i%mod==0:
            print(str(int(i/n*100))+"% ", end='')
    print('')
    fout.close()
    fout.close()
    flabel.close()

In [ ]:
#Converting train and test data to csv
convertbinMNIST("train-images-idx3-ubyte", "train-labels-idx1-ubyte",
        "fmnist_train.csv", 60000)
convertbinMNIST("t10k-images-idx3-ubyte", "t10k-labels-idx1-ubyte",
        "fmnist_test.csv", 10000)

In [ ]:
!wc fmnist_test.csv
!wc fmnist_train.csv

### Data preprocessing and validation

In [ ]:
#Import required packages
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import keras
from keras.layers import Dense, Flatten, Conv2D, Dropout, MaxPooling2D
import plotly.graph_objs as go
from plotly import subplots
from plotly.offline import iplot

In [ ]:
#Parameters
IMG_ROWS = 28
IMG_COLS = 28
CLASSES = 10
TEST_SIZE = 0.2
RANDOM_STATE = 2020
NO_EPOCHS = 20
BATCH_SIZE = 128

In [ ]:
#Load train and test data
train_data = pd.read_csv("fmnist_train.csv",header=None)
test_data = pd.read_csv("fmnist_test.csv",header=None)

In [ ]:
#Dimensions check
print("Fashion MNIST train -  rows:",train_data.shape[0]," columns:", train_data.shape[1])
print("Fashion MNIST test -  rows:",test_data.shape[0]," columns:", test_data.shape[1])
#Data loaded properly

**There are 10 different classes of images:**

* **0**: **T-shirt/top**;   
* **1**: **Trouser**;   
* **2**: **Pullover**;   
* **3**: **Dress**;
* **4**: **Coat**;
* **5**: **Sandal**;
* **6**: **Shirt**;
* **7**: **Sneaker**;
* **8**: **Bag**;
* **9**: **Ankle boot**.

In [ ]:
#Check labels distribution in training set
train_data[0].value_counts()

In [ ]:
#Check labels distribution in test set
test_data[0].value_counts()

In [ ]:
#Function - preparing data for DL model
def data_preproc(inp):
    out_y = keras.utils.to_categorical(inp[0], CLASSES)
    num_images = inp.shape[0]
    x_as_array = inp.values[:,1:]
    #Reshaping
    x_shaped = x_as_array.reshape(num_images, IMG_ROWS, IMG_COLS, 1)
    #Scaling
    out_x = x_shaped / 255
    return out_x, out_y

In [ ]:
X, y = data_preproc(train_data)
X_test, y_test = data_preproc(test_data)

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=TEST_SIZE, random_state=RANDOM_STATE)

In [ ]:
X_train.shape

In [ ]:
X_val.shape

### Build model specification

In [ ]:
# Build sequential model with Keras
model = keras.models.Sequential()
model.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu',
                 kernel_initializer='he_normal',
                 input_shape=(IMG_ROWS, IMG_COLS, 1)))
model.add(MaxPooling2D((2, 2)))
model.add(Dropout(0.25))
model.add(Conv2D(64, 
                 kernel_size=(3, 3), 
                 activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(Dropout(0.4))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(CLASSES, activation='softmax'))


model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer='adam',
              metrics=['accuracy'])

In [ ]:
model.summary()

In [ ]:
keras.utils.plot_model(model)

### Training model

In [ ]:
train_model = model.fit(X_train, y_train,
                  batch_size=BATCH_SIZE,
                  epochs=NO_EPOCHS,
                  verbose=1,
                  validation_data=(X_val, y_val))

In [ ]:
score = model.evaluate(X_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])
#Model achieved 90.8% accuracy on test set
#This is looking quite good compared to results on https://github.com/zalandoresearch/fashion-mnist
#and considering only 20 epochs were used

In [ ]:
#Functions - plotting model learning history
def create_trace(x,y,ylabel,color):
        trace = go.Scatter(
            x = x,y = y,
            name=ylabel,
            marker=dict(color=color),
            mode = "markers+lines",
            text=x
        )
        return trace
    
def plot_accuracy_and_loss(train_model):
    hist = train_model.history
    acc = hist['accuracy']
    val_acc = hist['val_accuracy']
    loss = hist['loss']
    val_loss = hist['val_loss']
    epochs = list(range(1,len(acc)+1))
    
    trace_ta = create_trace(epochs,acc,"Training accuracy", "Green")
    trace_va = create_trace(epochs,val_acc,"Validation accuracy", "Red")
    trace_tl = create_trace(epochs,loss,"Training loss", "Blue")
    trace_vl = create_trace(epochs,val_loss,"Validation loss", "Magenta")
   
    fig = subplots.make_subplots(rows=1,cols=2, subplot_titles=('Training and validation accuracy',
                                                             'Training and validation loss'))
    fig.append_trace(trace_ta,1,1)
    fig.append_trace(trace_va,1,1)
    fig.append_trace(trace_tl,1,2)
    fig.append_trace(trace_vl,1,2)
    fig['layout']['xaxis'].update(title = 'Epoch')
    fig['layout']['xaxis2'].update(title = 'Epoch')
    fig['layout']['yaxis'].update(title = 'Accuracy', range=[0,1])
    fig['layout']['yaxis2'].update(title = 'Loss', range=[0,1])

    
    iplot(fig, filename='accuracy-loss')

In [ ]:
plot_accuracy_and_loss(train_model)
#Accuracy and loss looks similar for both training and validation set - overfitting is addressed with Dropout

In [ ]:
#Checking fit metrics across all classes
predicted_classes = model.predict_classes(X_test)
y_true = test_data.iloc[:, 0]
labels = {0 : "T-shirt/top", 1: "Trousers", 2: "Pullover", 3: "Dress", 4: "Coat",
          5: "Sandal", 6: "Shirt", 7: "Sneaker", 8: "Bag", 9: "Ankle Boot"}
target_names = ["Class {} ({}) :".format(i,labels[i]) for i in range(CLASSES)]
print(classification_report(y_true, predicted_classes, target_names=target_names))

As f1-score combine both precision and recall let's focus on analyzing that metric.
By looking on f1-score we can see that some classes were predicted better than other.
1. Very well predicted classes (f1 >= 0.9):
    * 1-Trousers
    * 3-Dress
    * 5-Sandal
    * 7-Sneaker
    * 8-Bag
    * 9-Ankle Boot
2. Well predicted classes (0.8 =< f1 < 0.9):
    * 0-T-shirt
    * 2-Pullover
    * 4-Coat
3. Poorly predicted classes (f1 < 0.8):
    * 6-Shirt

Shirt is similar to T-shirt, so model may have problem with distingushing those two.

In [ ]:
#Save model for deployment
model.save('FMNIST_Model.h5')

### Deployment proposal
Saved Keras model can be incorporated into application and deployed for end users. For scalable deployment using one of the cloud providers e.g. Google Cloud Platform is one of reasonable optiom. Cloud solutions for app deployment are usually managed and can be scaled automatically based on set metric (CPU usage, requests per second). 

### App design

To expose model we can develop Flask app for developer-based approach (binary input). App files can be found in _keras-app_ folder:
* **Dockerfile** - docker definition file to containerize the app
* **requirements.txt** - Python dependencies to be loaded in Docker container
* **app.py** - Flask app that loads built keras model and wait for binary input on _localhost:5000/predict_ endpoint
* **FMNIST_Model.h5** - saved keras model for Fashion-MNIST classification
* **example_input** - binary file with one image to classify

### Local deployment (for development and validation)
I assume Docker is installed locally and it's running.

While in keras-app folder, we build Docker image, run container and validate if app is running using:
```sh
docker build -t keras-app:latest .
docker run -d -p 5000:5000 keras-app
docker ps -a
```
Below example request to the deployed app
```sh
curl -X POST -F image=@example_input 'http://localhost:5000/predict'
```
We should get JSON response:
```json
{
  "predictions": [
    {
      "label": 0,
      "probability": 0.9999948740005493
    },
    {
      "label": 1,
      "probability": 1.7362290262185748e-13
    },
    {
      "label": 2,
      "probability": 1.8749882428892306e-06
    },
    {
      "label": 3,
      "probability": 1.6042036588004294e-09
    },
    {
      "label": 4,
      "probability": 8.0264128676788e-11
    },
    {
      "label": 5,
      "probability": 9.621579596759606e-18
    },
    {
      "label": 6,
      "probability": 3.266337898821803e-06
    },
    {
      "label": 7,
      "probability": 5.098662132007302e-21
    },
    {
      "label": 8,
      "probability": 4.3966014856566815e-11
    },
    {
      "label": 9,
      "probability": 1.5432945262057027e-17
    }
  ],
  "success": true
}
```

### GKE deployment
Launch Google Cloud Shell in GCP Console and copy app files (preferably from private git repo)
```sh
git clone https://github.com/<some_user>/FMNIST-App)
cd FMNIST-App
```
Build Docker image with proper tag for Google Container Registry
```sh
docker build -t gcr.io/${PROJECT_ID}/FMNIST-app:v1 .
```
Push image to GCR (I assume shell docker is authenticated in GCR)
```sh
docker push gcr.io/${PROJECT_ID}/FMNIST-app:v1
```
Create cluster on GKE
```sh
gcloud container clusters create FMNIST-cluster --num-nodes=2
```
Deploy our app on GKE
```sh
kubectl create deployment FMNIST-web --image=gcr.io/${PROJECT_ID}/FMNIST-app:v1
```
Expose deployment to internet with Load Balancer
```sh
kubectl expose deployment FMNIST-web --type=LoadBalancer --port 80 --target-port 5000
```
We can validate and check availability of the service by running
```sh
kubectl get pods #Check running pods
kubectl get deployment FMNIST-web #Check deployment status
kubectl get service #Check LoadBalancer status and external IP
#Run on local machine
curl -X POST -F image=@example_input http://<EXTERNAL_LB_IP>/predict
```
To add autoscaling based on CPU utilization we can call
```sh
kubectl autoscale deployment FNIST-web --cpu-percent=80 --min=1 --max=10
```
To check information about launched Horizontal Pod Autoscaler (HPA) we can call
```sh
kubectl get hpa
```